> Convolution layer의 feature map의 크기를 줄일 때, pooling보다는 stride=2를 이용!

> F(x)+x할 때 서로의 차원의 크기가 맞지 않으면, 1x1 conv를 통해서 feature map의 갯수를 맞추자 



In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Activation, add, Flatten, GlobalAveragePooling2D, Reshape, multiply
from keras.models import Model

import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator


width = 32
height = 32

# cifar data download
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
y_train.shape

(50000, 1)

In [ ]:
num_classes = 10
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
val_images, val_labels = x_train[:500], y_train[:500]
train_images, train_labels = x_train[500:], y_train[500:]

### concat을 진행하는게 아니라 add를 진행하는 이유?
element by elemet 값을 더해주는 것이기 때문이다.

In [ ]:
def Residual_Block(x, filter):

    x_skip = x 
    f= filter

    x = Conv2D(f, kernel_size=(3,3), strides=1, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(f, kernel_size=(3,3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # projection 진행
    x_skip = Conv2D(f, kernel_size=(1,1),strides=(1,1),padding='same')(x_skip) #1x1 conv을 f만큼 진행해서 projection을 한다.
    x_skip = BatchNormalization()(x_skip)    

    x = add([x, x_skip]) # concat을 처음에 진행했는데 조금 다른 값이 나온거 같다. why?
    x = Activation('relu')(x)    
    
    return x


In [ ]:
def Residual_Block_50(x, filters):
    f1, f2 = filters
    x_skip = x # 초기의 x

    x = Conv2D(f1, kernel_size=(1,1), strides=1, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f1, kernel_size=(3,3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f2, kernel_size=(1,1), strides=1, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #scale
    x_skip = Conv2D(f2, kernel_size=(1,1),strides=(1,1),padding='same')(x_skip)
    x_skip = BatchNormalization()(x_skip)    

    #skip connection
    x = add([x, x_skip])
    x = Activation('relu')(x)    
    
    return x

In [ ]:
inputs = Input(shape=(32,32,3))
x = Conv2D(64, kernel_size=7, strides=2, padding='valid')(inputs)
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

for i in range(3):
  x = Residual_Block_50(x, filters=(64,256)) 

for i in range(4):
  x = Residual_Block_50(x, filters =(128,512)) 

for i in range(6):
  x = Residual_Block_50(x, filters =(256,1024)) 

for i in range(3):
  x = Residual_Block_50(x, filters =(512,2048)) 

x = GlobalAveragePooling2D()(x) 

outputs = Dense(10, activation = 'softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 13, 13, 64)   9472        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 7, 7, 64)     0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 7, 64)     4160        ['max_pooling2d[0][0]']          
                                                                                              

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagenerator = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
                                   zoom_range = 0.2, horizontal_flip = True)
validation_datagenerator = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
                                   zoom_range = 0.2, horizontal_flip = True)

In [ ]:
history = model.fit(train_datagenerator.flow(train_images,train_labels, batch_size = 32), 
                    validation_data = validation_datagenerator.flow(val_images, val_labels, batch_size = 32),
                    epochs = 10)

Epoch 1/10
1547/1547 [==============================] - 234s 140ms/step - loss: 1.9411 - accuracy: 0.2984 - val_loss: 1.6912 - val_accuracy: 0.3740
Epoch 2/10
1547/1547 [==============================] - 214s 138ms/step - loss: 1.5513 - accuracy: 0.4356 - val_loss: 1.4767 - val_accuracy: 0.4840
Epoch 3/10
1547/1547 [==============================] - 214s 138ms/step - loss: 1.3949 - accuracy: 0.4957 - val_loss: 1.4469 - val_accuracy: 0.4900
Epoch 4/10
1547/1547 [==============================] - 214s 138ms/step - loss: 1.2768 - accuracy: 0.5453 - val_loss: 1.4827 - val_accuracy: 0.4540
Epoch 5/10
1547/1547 [==============================] - 214s 138ms/step - loss: 1.1777 - accuracy: 0.5830 - val_loss: 1.1616 - val_accuracy: 0.5980
Epoch 6/10
1547/1547 [==============================] - 213s 138ms/step - loss: 1.0894 - accuracy: 0.6162 - val_loss: 1.0607 - val_accuracy: 0.6200
Epoch 7/10
1547/1547 [==============================] - 214s 138ms/step - loss: 1.0146 - accuracy: 0.6448 - val_

BasicBlock (ResNet18, ResNet34)이랑 Bottleneck (ResNet50, ResNet101, ResNet152)

```
# for i in range(3):
  x = Residual_Block_50(x, filters=(64,256)) 

for i in range(4):
  x = Residual_Block_50(x, filters =(128,512)) 

for i in range(6):
  x = Residual_Block_50(x, filters =(256,1024)) 

for i in range(3):
  x = Residual_Block_50(x, filters =(512,2048)) 
```
ResNet-50으로 정의된 내용을 위의 내용으로 대체해서 쓰면 basic으로 바뀐다.


In [ ]:
x = Residual_Block(x, filter = 64)
x = Residual_Block(x, filter = 64)

x = Residual_Block(x, filter = 128)
x = Residual_Block(x, filter = 128)

x = Residual_Block(x, filter = 256)
x = Residual_Block(x, filter = 256)

x = Residual_Block(x, filter = 512)
x = Residual_Block(x, filter = 512)